In [3]:
import os
from openai import OpenAI
import pandas as pd

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def generate_tweet(label):
    prompt = (
        "Here are some examples of tweets about COVID-19 vaccination and their stances:\n"
        "Tweet: people keep asking for magic dust. it doesn't exist”, says @drmikeryan..“only magic dust we do have is vaccination. problem is, we're not sprinkling that evenly around the world, and we're working against ourselves.\n"
        "Label: in-favor\n\n"
        "Tweet: big news: 60 seattle bars and restaurants now require proof of vaccination to eat on their premises."
        "Label: neutral-or-unclear\n\n"
        "Tweet: makes ya question everything! vaccine passports obsolete, vaxxers questioning what they have put in them, why the push to get it into children?\n"
        "Label: against\n\n"
        "Now generate a very creative tweet about COVID-19 vaccination. "
        f"Make sure the stance of the tweet is '{label}'. Provide a creative tweet that matches the stance. "
        "Tweet: <tweet> Label: <label>"
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a creative person that likes to tweet about COVID-19 vaccination. You are going to come up with new tweets every time which varies in length."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.9,  
        top_p=0.95
    )
    content = response.choices[0].message.content.strip()
    tweet = content.split("Tweet: ")[1].split(" Label:")[0].strip()
    label = content.split("Label: ")[1].strip()
    return tweet, label

n_samples_per_label = 500
tweets = []
labels = []

stances = ['in-favor', 'against', 'neutral-or-unclear']

for stance in stances:
    for _ in range(n_samples_per_label):
        tweet, label = generate_tweet(stance)
        tweets.append(tweet)
        labels.append(label)

df = pd.DataFrame({
    'tweet': tweets,
    'label_true': labels
})
df.to_csv('GAItweets.csv', index=False)
